# Rayleigh-Benard Convection

Within the present task it is necessary to solve a Navier-Stokes equations (NSE) to  describe the transport of an incompressible fluid. The non-dimensional NSE consist of:

- Mass conservation equation:  

$$
\frac{\partial \~u}{\partial \~x} + \frac{\partial \~v}{\partial \~y} 
$$

- Momentum conservation equations

$$\frac{\partial \tilde{u}}{\partial \tilde{t}} + \tilde{u} \frac{\partial \tilde{u}}{\partial \tilde{x}} + \tilde{v} \frac{\partial \tilde{u}}{\partial \tilde{y}} = -\frac{\partial \tilde{p}}{\partial \tilde{x}} + \frac{1}{Re} \left( \frac{\partial^2 \tilde{u}}{\partial \tilde{x}^2} + \frac{\partial^2 \tilde{u}}{\partial \tilde{y}^2} \right)$$

$$\frac{\partial \tilde{v}}{\partial \tilde{t}} + \tilde{u} \frac{\partial \tilde{v}}{\partial \tilde{x}} + \tilde{v} \frac{\partial \tilde{v}}{\partial \tilde{y}} = -\frac{\partial \tilde{p}}{\partial \tilde{y}} + \frac{1}{Re} \left( \frac{\partial^2 \tilde{v}}{\partial \tilde{x}^2} + \frac{\partial^2 \tilde{v}}{\partial \tilde{y}^2} \right) + \tilde{\rho}g$$

Where:

- $\tilde{x}$ is the non-dimensional space component.

- $\tilde{t}$ is the non-dimensional time.

- $\tilde{u}, \tilde{v}$ are the horizontal and vertical components of the non-dimensional velocity vector.

- $\tilde{p}$ is the non-dimensional pressure.

- $Re$ is the Reynolds number, defined as $Re = u_{ref}x_{ref} / \nu$, where $u_{ref}$ is the reference velocity, $x_{ref}$ is the reference length scale, and $\nu$ is the kinematic viscosity of the fluid.

- $\tilde{\rho}$ is the non-dimensional density of the fluid, the value of which is governed by the following scalar transport equation: The evolution of the non-dimensional density (or temperature scalar) is governed by:

$$\frac{\partial \tilde{\rho}}{\partial \tilde{t}} + \tilde{u} \frac{\partial \tilde{\rho}}{\partial \tilde{x}} + \tilde{v} \frac{\partial \tilde{\rho}}{\partial \tilde{y}} = \frac{1}{Pe} \left( \frac{\partial^2 \tilde{\rho}}{\partial \tilde{x}^2} + \frac{\partial^2 \tilde{\rho}}{\partial \tilde{y}^2} \right)$$

Where:

- $Pe$ is the Peclet number, defined as $Pe = u_{ref}x_{ref} / D$.
- $D$ corresponds to the diffusion coefficient.

Scenario: Solve the NSE for a heated lower wall below a cold top wall together with the  convection-diffusion equation for non-dimensional density using the SIMPLE algorithm  within a 2D-squared domain, where $L_X = L_Y = 1[cm]$, the kinematic viscosity of the fluid  $v = 10^{-6}[m^2/s]$ and the diffusion coefficient $D = 10^{-8}[m^2/s]$. 

Boundary conditions. For velocity and pressure fields the boundary conditions are no-slip walls  (i.e. zero values of velocity and zero gradient for pressure). For non-dimensional density: top  wall $\rho  = 1$, bottom wall $\rho  = 0$, and periodic BC for side walls.   

Initial conditions. Pressure and velocity are initialized with zero values. Non-dimensional  density initialized with values $\rho  = 0$.  

In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from typing import Dict, Tuple
import matplotlib.pyplot as plt
from dataclasses import dataclass

1. Grid Generation (Staggered Grid)
In a staggered grid, we don't just have one set of coordinates. We define separate locations for the cell centers (Scalars) and the cell faces (Velocities).

In [4]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

In [11]:
# Parameters from documentation
L = 0.01  # 1 cm
Nx, Ny = 40, 40
dx, dy = L/Nx, L/Ny
dt = 0.001 

# Grid Coordinates
x_p = np.linspace(dx/2, L-dx/2, Nx) # Cell centers
y_p = np.linspace(dy/2, L-dy/2, Ny)

2. Matrix Assembly Logic

For each variable, we flatten the 2D grid into a 1D vector of size $N = N_x \times N_y$. The mapping is usually $k = i + j \cdot N_x$.A. Periodic Boundary Conditions (Mapping)Since the side walls are periodic, the "West" neighbor of the first column ($i=0$) is the last column ($i=Nx-1$).

In [10]:
def get_k(i, j):
    """Maps 2D indices to 1D with periodic X-boundaries."""
    return (i % Nx) + j * Nx

3. Step-by-Step Numerical Implementation

Step 1: Solve for Density ($\tilde{\rho}$)

We solve $A_\rho \rho = b_\rho$. The matrix $A_\rho$ is constructed using the current velocity field.

Starting from

Converted to


$$\frac{\rho_{i,j}^{k+1} - \rho_{i,j}^k}{\Delta t} + u \frac{\rho_{i+1,j}^{k+1} - \rho_{i-1,j}^{k+1}}{2\Delta x} + v \frac{\rho_{i,j+1}^{k+1} - \rho_{i,j-1}^{k+1}}{2\Delta y} = \frac{1}{Pe} \left( \frac{\rho_{i+1,j}^{k+1} - 2\rho_{i,j}^{k+1} + \rho_{i-1,j}^{k+1}}{\Delta x^2} + \frac{\rho_{i,j+1}^{k+1} - 2\rho_{i,j}^{k+1} + \rho_{i,j-1}^{k+1}}{\Delta y^2} \right)$$

Then by saying {i, j} = p, {i + 1, j} = e, {i - 1, j} = w, {i, j+1} = n, {i, j - 1} = s and $\alpha = \frac{u \Delta t}{2\Delta x}$, $\beta = \frac{v \Delta t}{2\Delta y}$ and $\alpha^* = \frac{u \Delta t}{Pe \Delta x}$, $\beta^* = \frac{v \Delta t}{Pe \Delta y}$



it ends in

$$
\rho_p^{k+1} * (1 - 2\alpha - 2\beta) + \rho_e^{k+1} (\alpha - \alpha^*) + \rho_w^{k+1} (\alpha - \alpha^*) + \rho_n^{k+1} ( \beta - \beta^*) + \rho_s^{k+1} (- \beta - \beta^*) = \rho^k_p
$$

In [ ]:
def solve_rho(u, v, rho_n, Pe, dt):
    N = Nx * Ny
    A = [] # Data for CSR
    
    # velocity components
    u0 = np.zeros_like(x_p)
    v0 = np.zeros_like(x_p)

    row = []
    col = []
    b = np.zeros(N)
    
    diff_coeff = 1 / (Pe * dx**2) # Assuming dx = dy

    for j in range(Ny):
        for i in range(Nx):
            k = get_k(i, j)
            
            # Neighbors (Periodic in X)
            ke, kw = get_k(i+1, j), get_k(i-1, j)
            kn, ks = get_k(i, j+1), get_k(i, j-1)
            
            # Coefficients (Central Difference)
            ae = diff_coeff - (u[i+1, j] / (2 * dx))
            aw = diff_coeff + (u[i, j] / (2 * dx))
            an = diff_coeff - (v[i, j+1] / (2 * dy))
            as_ = diff_coeff + (v[i, j-1] / (2 * dy))
            
            ap = (1/dt) + (2*diff_coeff/dx**2 + 2*diff_coeff/dy**2)
            
            # Boundary Conditions for Density: Top=1, Bottom=0
            if j == Ny - 1: # Top wall
                b[k] = (rho_n[k]/dt) + an * 1.0
                ap += an # Adjust diagonal for Dirichlet
            elif j == 0: # Bottom wall
                b[k] = (rho_n[k]/dt) + as_ * 0.0
                ap += as_
            else:
                # Internal nodes
                A.extend([-ae, -aw, -an, -as_])
                col.extend([ke, kw, kn, ks])
                row.extend([k, k, k, k])
                b[k] = rho_n[k]/dt
                
            A.append(ap)
            col.append(k)
            row.append(k)
            
    rho_matrix = csr_matrix((A, (row, col)), shape=(N, N))
    return spsolve(rho_matrix, b)